# Librerias

In [1]:
random_state = 43992294
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MultiLabelBinarizer

from xgboost import XGBClassifier
from sklearn.preprocessing import FunctionTransformer

import ast

import warnings

from tqdm.auto import tqdm

from sklearn.feature_selection import RFECV

from sklearn.model_selection import StratifiedKFold, cross_val_score

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Fuciones

In [5]:
def limpiar_y_agrupar(df, columna, umbral=100):
    """
    Limpia y agrupa categorías en una columna específica de un DataFrame.
    Las categorías con frecuencia menor que el umbral se agrupan en 'Otros_{columna}',
    y los valores NaN se reemplazan por 'Desconocidos_{columna}'.

    Parameters:
    - df: DataFrame de pandas.
    - columna: Nombre de la columna a limpiar y agrupar.
    - umbral: Número mínimo de observaciones para mantener una categoría.

    Returns:
    - df: DataFrame con la columna modificada.
    """
    # Contar la frecuencia de cada categoría incluyendo NaN
    conteo_categorias = df[columna].value_counts(dropna=False)
    
    # Identificar categorías con observaciones por debajo del umbral
    categorias_pequenas = conteo_categorias[conteo_categorias < umbral].index

    # Filtrar las categorías poco frecuentes excluyendo NaN
    categorias_pequenas = [cat for cat in categorias_pequenas if pd.notna(cat)]
    
    # Crear valor de relleno para categorías poco frecuentes
    valor_pequeno = 'Otros'

    # Crear valor de relleno para NaNs
    valor_desconocido = 'Desconocidos'

    # Aplicar la asignación de categorías
    df[columna] = df[columna].apply(
        lambda x: valor_pequeno if x in categorias_pequenas 
                  else (valor_desconocido if pd.isna(x) else x)
    )
    
    return df

def cramers_v(confusion_matrix):
    """
    Calcula la estadística de Cramér's V para medir la asociación entre dos variables categóricas.

    Parameters:
    - confusion_matrix: Matriz de confusión (tabla de contingencia) entre dos variables.

    Returns:
    - Cramér's V: Valor entre 0 y 1 que indica la fuerza de la asociación.
    """
    # Calcular el estadístico chi-cuadrado
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    # Número total de observaciones
    n = confusion_matrix.sum().sum()
    # Obtener el número de filas y columnas de la matriz de confusión
    r, k = confusion_matrix.shape
    if min(r, k) == 1:
        return np.nan  # Evitar dividir por cero
    # Calcular Cramér's V
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))


def expand_list_dummies(df, column, delimiter='|', prefix=None, suffix=None):
    """
    Expande una columna que contiene listas (o cadenas representando listas) en múltiples columnas binarias
    utilizando pandas.get_dummies, una por cada categoría única.

    Parameters:
    - df (pd.DataFrame): DataFrame de pandas.
    - column (str): Nombre de la columna a expandir.
    - delimiter (str): Delimitador a usar en get_dummies. Por defecto es '|'.
    - prefix (str, optional): Prefijo para las nuevas columnas binarias.
    - suffix (str, optional): Sufijo para las nuevas columnas binarias.

    Returns:
    - categories (set): Conjunto de categorías únicas encontradas.
    - df_expanded (pd.DataFrame): DataFrame con las nuevas columnas binarias añadidas y la columna original eliminada.
    """
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    
    # Definir el valor para valores faltantes
    valor_desconocido = f'Desconocidos_{column}'
    
    # Reemplazar NaN por el valor desconocido
    df[column] = df[column].fillna(f'["{valor_desconocido}"]')
    
    # Convertir las cadenas que representan listas en listas reales de Python
    def parse_list(x):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return parsed
            else:
                return [x]
        except (ValueError, SyntaxError):
            return [x]
    
    df[column] = df[column].apply(parse_list)
    
    # Convertir listas en cadenas separadas por el delimitador
    df['temp_combined'] = df[column].apply(lambda lst: delimiter.join(map(str, lst)))
    
    # Generar las columnas binarias usando get_dummies
    dummies = df['temp_combined'].str.get_dummies(sep=delimiter)
    
    # Añadir prefijo y/o sufijo si se especifica
    if prefix:
        dummies = dummies.add_prefix(f"{prefix}_")
    if suffix:
        dummies = dummies.add_suffix(f"_{suffix}")
    
    # Concatenar las columnas binarias al DataFrame original
    df_expanded = pd.concat([df.drop(columns=[column, 'temp_combined']), dummies], axis=1)
    
    # Obtener las categorías únicas
    categories = set(dummies.columns)
    
    return categories, df_expanded


def categorizar_hora(hora):
    """
    Categoriza una hora dada en 'mañana', 'tarde' o 'noche'.

    Parameters:
    - hora: Cadena de tiempo en formato 'HH:MM:SS'.

    Returns:
    - Categoría de la parte del día: 'mañana', 'tarde' o 'noche'.
    """
    # Extraer la hora como entero
    hora = int(hora.split(':')[0])
    if 0 <= hora < 12:
        return 'mañana'
    elif 12 <= hora < 18:
        return 'tarde'
    else:
        return 'noche'


def convertir_auction_time(df):
    """
    Convierte la columna 'auction_time' de formato timestamp a categorías de parte del día ('mañana', 'tarde', 'noche')
    y crea columnas binarias para cada categoría.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - Lista de categorías creadas.
    - df: DataFrame con las nuevas columnas binarias añadidas y las columnas originales eliminadas.
    """
    # Convertir 'auction_time' de timestamp a cadena de tiempo 'HH:MM:SS'
    df['auction_time'] = pd.to_datetime(df['auction_time'], unit='s').dt.strftime('%H:%M:%S')
    
    # Crear una nueva columna con las categorías de parte del día
    df['parte_del_dia'] = df['auction_time'].apply(categorizar_hora)
    
    # Crear columnas binarias para cada parte del día
    df['mañana'] = (df['parte_del_dia'] == 'mañana').astype(int)
    df['tarde'] = (df['parte_del_dia'] == 'tarde').astype(int)
    df['noche'] = (df['parte_del_dia'] == 'noche').astype(int)
    
    # Eliminar las columnas intermedias si no se necesitan
    df.drop(['parte_del_dia', 'auction_time'], axis=1, inplace=True)
    
    return ['mañana', 'tarde', 'noche'], df


def IntFloatToStr(df):
    """
    Limpia los datos del DataFrame convirtiendo cualquier columna que contenga una combinación de 
    strings con enteros o strings con floats a cadenas de texto. Los valores NaN se reemplazan por '0'.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - df: DataFrame limpio con las columnas convertidas a strings donde corresponda.
    """
    # Recorrer todas las columnas del DataFrame
    for column in tqdm(df.columns, desc="Limpiando datos"):
        unique_types = df[column].apply(type).unique()
        
        # Verificar si la columna contiene tanto strings como enteros o floats
        if {str, int}.issubset(set(unique_types)) or {str, float}.issubset(set(unique_types)):
            # Convertir todos los valores a string, reemplazando NaN por '0'
            df[column] = df[column].apply(lambda x: str(x) if pd.notna(x) else '0')
    
    return df


def data_cleaning(df):
    """
    Realiza la limpieza y preprocesamiento de un DataFrame siguiendo varios pasos:
    - Convertir y expandir columnas específicas.
    - Agrupar categorías poco frecuentes en columnas categóricas.
    - Limpiar los datos finales.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - all_categories: Conjunto de todas las categorías únicas encontradas.
    - df: DataFrame limpio y preprocesado.
    """

    df = df.drop(columns=['device_id'])
    
    all_categories = set()
    
    # Mostrar el tamaño inicial del DataFrame
    print(f"Paso inicial: Tamaño del DataFrame: {df.shape}")

    # Identificar columnas categóricas excluyendo algunas específicas
    categorical_features = [
        col for col in df.select_dtypes(include=['object']).columns.tolist()
        if col not in ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']
    ]
    print(f"Paso 1: Columnas categóricas identificadas (excluidas algunas): {categorical_features}")

    # Agrupar categorías poco frecuentes en las columnas categóricas
    for col in tqdm(categorical_features, desc="Agrupando categorías poco frecuentes"):
        print(f"Procesando columna categórica: {col}")
        df = limpiar_y_agrupar(df, col, umbral=100)

    # Mostrar el estado del DataFrame después de agrupar categorías poco frecuentes
    print(f"Paso 2: Tamaño del DataFrame después de agrupar categorías: {df.shape}")

    # Convertir y expandir la columna 'auction_time'
    print(f"Paso 3: Expandiendo la columna 'auction_time'")
    categories_time, df = convertir_auction_time(df)
    all_categories.update(categories_time)
    print(f"Categorías extraídas de 'auction_time': {categories_time}")

    # Expandir la columna 'auction_list_0' usando expand_list_dummies
    print(f"Paso 4: Expandiendo la columna 'auction_list_0'")
    categories_auction, df = expand_list_dummies(
        df, 
        column='auction_list_0', 
        delimiter='|', 
        prefix=None, 
        suffix='auction_list_0'
    )
    all_categories.update(categories_auction)
    print(f"Categorías extraídas de 'auction_list_0': {categories_auction}")

    # Expandir la columna 'action_list_1' usando expand_list_dummies
    print(f"Paso 5: Expandiendo la columna 'action_list_1'")
    categories_action1, df = expand_list_dummies(
        df, 
        column='action_list_1', 
        delimiter='|', 
        prefix=None, 
        suffix='action_list_1'
    )
    all_categories.update(categories_action1)
    print(f"Categorías extraídas de 'action_list_1': {categories_action1}")

    # Expandir la columna 'action_list_2' usando expand_list_dummies
    print(f"Paso 6: Expandiendo la columna 'action_list_2'")
    categories_action2, df = expand_list_dummies(
        df, 
        column='action_list_2', 
        delimiter='|', 
        prefix=None, 
        suffix='action_list_2'
    )
    all_categories.update(categories_action2)
    print(f"Categorías extraídas de 'action_list_2': {categories_action2}")

    # Mostrar el tamaño del DataFrame antes de la limpieza adicional
    print(f"Paso 7: Tamaño del DataFrame antes de la limpieza adicional: {df.shape}")

    # Realizar limpieza adicional de los datos
    df = IntFloatToStr(df)

    # Tamaño final del DataFrame
    print(f"Paso 8: Tamaño final del DataFrame: {df.shape}")

    return all_categories, df

def ajustar_columnas_test(train_df, test_df):
    """
    Asegura que las columnas del conjunto de test coincidan con las del conjunto de entrenamiento,
    con la excepción de que 'Label' no se puede borrar de train y 'id' no se puede borrar de test.
    
    Si faltan columnas en el conjunto de test, se agregan con valor 0.
    Si sobran columnas en el conjunto de test que no están en el de entrenamiento, se eliminan.
    
    Parameters:
    - train_df: DataFrame del conjunto de entrenamiento (con las columnas transformadas)
    - test_df: DataFrame del conjunto de test (sin transformar)
    
    Returns:
    - test_df: DataFrame del conjunto de test con las columnas ajustadas.
    """
    # Encontrar las columnas que están en train pero no en test, excepto 'Label'
    missing_cols = set(train_df.columns) - set(test_df.columns) - {'Label'}

    # Encontrar las columnas que están en test pero no en train, excepto 'id'
    extra_cols = set(test_df.columns) - set(train_df.columns) - {'id'}

    # Agregar las columnas faltantes en el conjunto de test con valor 0
    for col in missing_cols:
        test_df[col] = 0

    # Eliminar las columnas extra del conjunto de test, excepto 'id'
    test_df = test_df.drop(columns=extra_cols)

    # Asegurarse de que las columnas estén en el mismo orden que en el conjunto de entrenamiento
    test_df = test_df[train_df.columns.difference(['Label']).tolist() + ['id']]

    return test_df

def ajustar_columnas_val(train_df, val_df):
    # Encontrar las columnas que están en train pero no en test, excepto 'Label'
    missing_cols = set(train_df.columns) - set(val_df.columns)

    # Encontrar las columnas que están en test pero no en train, excepto 'id'
    extra_cols = set(val_df.columns) - set(train_df.columns)

    # Agregar las columnas faltantes en el conjunto de test con valor 0
    for col in missing_cols:
        val_df[col] = 0

    # Eliminar las columnas extra del conjunto de test, excepto 'id'
    val_df = val_df.drop(columns=extra_cols)

    return val_df

# Función para manejar None en hiperparámetros en el nombre del archivo
def handle_none(value):
    return 'None' if value is None else str(value)


def concatenar(df1, df2):
    # Encontrar las columnas que están en df1 pero no en df2
    missing_cols = set(df1.columns) - set(df2.columns)

    # Encontrar las columnas que están en df2 pero no en df1
    extra_cols = set(df2.columns) - set(df1.columns)

    # Añadir las columnas faltantes en df2 con valores 0
    for col in missing_cols:
        df2[col] = 0

    # Añadir las columnas faltantes en df1 con valores 0
    for col in extra_cols:
        df1[col] = 0

    # Asegurarse de que ambas tengan las mismas columnas en el mismo orden
    df1 = df1[df2.columns]

    # Concatenar los DataFrames
    df_concat = pd.concat([df1, df2], ignore_index=True)

    return df_concat

In [6]:
def corregir_agrupaciones(df):
    """
    Corrige las agrupaciones realizadas por la función limpiar_y_agrupar.
    Cambia los valores que contienen 'Otros_{columna}' por 'Otros' y 
    'Desconocidos_{columna}' por 'Desconocidos' en todas las columnas del DataFrame.

    Parameters:
    - df: DataFrame de pandas.

    Returns:
    - df: DataFrame con las columnas corregidas.
    """
    for columna in df.columns:
        if df[columna].dtype == 'object':  # Aplicar solo a columnas categóricas
            df[columna] = df[columna].replace(
                {f'Otros_{columna}': 'Otros', f'Desconocidos_{columna}': 'Desconocidos'}
            )
    return df

def trucazo(test, train):
    # 1. Eliminar columnas de train que no están en test, excepto 'Label'
    extra_cols = set(train.columns) - set(test.columns) - {'Label'}
    train = train.drop(columns=extra_cols)

    # 2. Agregar columnas faltantes en train con valor por defecto basado en su tipo de dato
    missing_cols = set(test.columns) - set(train.columns)
    for col in missing_cols:
        if col != 'Label':  # Asegurarte de no agregar 'Label' si falta
            if test[col].dtype == 'object':
                train[col] = 'Otros'  # Si es categórica, asignar 'Otros'
            else:
                train[col] = 0  # Si es numérica, asignar 0
    
    # 3. Reemplazar valores no comunes sólo en las columnas categóricas (str)
    for col in train.columns:
        if col in test.columns and train[col].dtype == 'object' and col != 'Label':  # Solo columnas categóricas
            # Identificar los valores únicos que están en train pero no en test
            valores_no_comunes = set(train[col].unique()) - set(test[col].unique())
            if len(valores_no_comunes) > 0:  # Si hay valores no comunes
                train[col] = train[col].apply(lambda x: 'Otros' if x in valores_no_comunes else x)

    return train


# Datasets

In [7]:
# train_data_20 = pd.read_csv('data/ctr_20.csv')
train_data = pd.read_csv('train_data_combined.csv')

test = pd.read_csv('test_data_cleaned.csv', dtype={'action_categorical_6': str})

In [8]:
print("Columnas de test antes:", test.shape[1])
print("Columnas de train antes:", train_data_20.shape[1])

train_data_20 = trucazo(test, train_data_20)

print("Columnas de test después:", test.shape[1])
print("Columnas de train después:", train_data_20.shape[1])

Columnas de test antes: 391
Columnas de train antes: 461
Columnas de test después: 391
Columnas de train después: 392


In [10]:
train_data_combined = train_data_20

In [27]:
categorical_features = train_data_combined.select_dtypes(include=['object']).columns

In [32]:
# Crear un diccionario para almacenar las categorías desconocidas por columna
categorias_desconocidas = {}

# Iterar a través de cada columna categórica
for columna in categorical_features:
    # Obtener las categorías únicas en el conjunto de entrenamiento
    categorias_train = set(train_data_20[columna].dropna().unique())
    
    # Obtener las categorías únicas en el conjunto de prueba
    categorias_test = set(test[columna].dropna().unique())
    
    # Identificar las categorías en test que no están en train
    desconocidas = categorias_test - categorias_train
    
    # Almacenar las categorías desconocidas en el diccionario como una lista
    categorias_desconocidas[columna] = list(desconocidas)
    
# Mostrar el diccionario completo de categorías desconocidas
print("\nDiccionario de categorías desconocidas por columna:")
for columna, categorias in categorias_desconocidas.items():
    print(f"'{columna}': {categorias}")



Columna 'action_categorical_0' tiene 2 categorías desconocidas:
  - dd6020ea
  - 2ab1d5f3

Columna 'action_categorical_1' tiene 3 categorías desconocidas:
  - 82f8fd50
  - ad4c0eac
  - 103788ed

Columna 'action_categorical_2' tiene 14 categorías desconocidas:
  - 75f72165
  - 2747ef98
  - fbd3cd06
  - ce64496e
  - fc3c6221
  - eafe914f
  - ea5b53ec
  - 4ff3d978
  - 342311c1
  - 93611a3c
  - 6f1bb457
  - 810d5ae8
  - f701b765
  - 5f2b3eb9

Columna 'action_categorical_3' tiene 5 categorías desconocidas:
  - eb5d06d9
  - 9726eea4
  - a7d9c64c
  - abc2ddca
  - 0f17b94c

Columna 'action_categorical_4' tiene 89 categorías desconocidas:
  - ec8cf03b
  - 3df97742
  - 0e645d6b
  - a999e8af
  - 30306bf1
  - 4e040950
  - 90240f9a
  - ae5bc59c
  - 001c9a66
  - 2f24de39
  - f225d5e3
  - a789c7fd
  - 6506eb50
  - 9a52c92a
  - 7b0ab77d
  - a5b67cdd
  - fa63bd4d
  - 3ab5b545
  - 12d13ec9
  - b0d40c3f
  - f195941c
  - 48eb1ab5
  - 80af0f18
  - 8c668b31
  - b1d83474
  - 3b148f0b
  - f40cfd11
  - c72738

# Entrenar

In [11]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Asegurarse de que X_train y X_val sean dataframes de pandas
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

In [15]:
# Definir las columnas categóricas y numéricas
categorical_features = X_train.select_dtypes(include=['object']).columns
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocesador común para imputación y codificación
common_preprocessor = ColumnTransformer(
    transformers=[
        # Imputar valores numéricos con la media
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        
        # Imputar valores categóricos con 'Desconocido' y aplicar One-Hot Encoding
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=True, handle_unknown='ignore'))
        ]), categorical_features)
    ],
    remainder='drop'  # Excluir columnas no especificadas
)

model_rf = RandomForestClassifier(random_state=random_state)

# Definir el modelo XGBoost
model_xgb = XGBClassifier(
    random_state=random_state, 
    use_label_encoder=False, 
    eval_metric='auc'
)

# Pipeline para XGBoost (mantiene matrices dispersas)
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('classifier', model_xgb),
])

# Pipeline para Random Forest (convierte a denso y optimiza tipos de datos)
pipeline_rf = Pipeline(steps=[
    ('preprocessor', common_preprocessor),
    ('to_dense', FunctionTransformer(lambda x: x.tocsr().astype('float32').toarray(), accept_sparse=True)),
    ('classifier', model_rf),
])

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [20]:
# Definir el espacio de búsqueda para los hiperparámetros de XGBoost
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)), 
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]), 
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 2),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),  # L1 regularización
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),  # L2 regularización
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),  # Muestreo a nivel de split
    'colsample_bynode': hp.uniform('colsample_bynode', 0.5, 1),  # Muestreo por nodo
    'grow_policy': hp.choice('grow_policy', ['depthwise', 'lossguide']),  # Estrategia de crecimiento
    'tree_method': hp.choice('tree_method', ['auto', 'approx', 'hist'])  # Métodos de construcción del árbol
}

# Función objetivo para Hyperopt utilizando validación cruzada
def objective_xgb(params):
    
    # Asegurar que los parámetros sean del tipo correcto
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    
    # Definir el modelo con los hiperparámetros actuales
    model_xgb = XGBClassifier(
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        n_estimators=params['n_estimators'],
        gamma=params['gamma'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        reg_alpha=params['reg_alpha'],  # L1 regularización
        reg_lambda=params['reg_lambda'],  # L2 regularización
        colsample_bylevel=params['colsample_bylevel'],  # Muestreo a nivel de split
        colsample_bynode=params['colsample_bynode'],  # Muestreo por nodo
        grow_policy=params['grow_policy'],  # Estrategia de crecimiento
        tree_method=params['tree_method'],  # Método de construcción del árbol
        use_label_encoder=False,  # Evitar advertencias en versiones más recientes de XGBoost
        eval_metric='auc',
        random_state=random_state
    )
    
    # Crear el pipeline
    pipeline_xgb = Pipeline(steps=[
        ('preprocessor', common_preprocessor),
        ('classifier', model_xgb),
    ])
    
    # Definir la validación cruzada
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    # Calcular el AUC utilizando validación cruzada
    auc = cross_val_score(pipeline_xgb, X_train, y_train, cv=cv, scoring='roc_auc').mean()
    
    # Opcional: imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")
    
    # Retornar el valor a minimizar (1 - AUC)
    return {'loss': 1 - auc, 'status': STATUS_OK}

In [21]:
# Ejecutar la optimización
trials = Trials()

best_xgb = fmin(
    fn=objective_xgb,
    space=space,
    algo=tpe.suggest,
    max_evals=2,
    trials=trials,
    rstate=np.random.default_rng(random_state)  # Asegurar reproducibilidad
)

# No es necesario volver a mapear los hiperparámetros aquí, ya se hizo dentro de la función objetivo
print("Mejores hiperparámetros para XGBoost:")
print(best_xgb)

  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data

Hiperparámetros: {'colsample_bylevel': 0.7846542046759237, 'colsample_bynode': 0.8602953305997507, 'colsample_bytree': 0.6615659784543418, 'gamma': 0.10922672283247337, 'grow_policy': 'lossguide', 'learning_rate': 0.1993070315803166, 'max_depth': 6, 'min_child_weight': 7, 'n_estimators': 100, 'reg_alpha': 0.32213467759367453, 'reg_lambda': 0.8036297375321274, 'scale_pos_weight': 1.4841116094332476, 'subsample': 0.8064682844784123, 'tree_method': 'approx'}, AUC: 0.8525
 50%|█████     | 1/2 [08:05<08:05, 485.55s/trial, best loss: 0.14753041032041292]

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data

Hiperparámetros: {'colsample_bylevel': 0.9423163809512285, 'colsample_bynode': 0.8633931980345834, 'colsample_bytree': 0.9732979141097506, 'gamma': 0.434258553578369, 'grow_policy': 'depthwise', 'learning_rate': 0.09934321000540317, 'max_depth': 4, 'min_child_weight': 9, 'n_estimators': 300, 'reg_alpha': 0.05928283380153099, 'reg_lambda': 0.08244095577728672, 'scale_pos_weight': 0.9923454624997129, 'subsample': 0.7031458153478625, 'tree_method': 'auto'}, AUC: 0.8483
100%|██████████| 2/2 [30:44<00:00, 922.49s/trial, best loss: 0.14753041032041292]
Mejores hiperparámetros para XGBoost:
{'colsample_bylevel': 0.7846542046759237, 'colsample_bynode': 0.8602953305997507, 'colsample_bytree': 0.6615659784543418, 'gamma': 0.10922672283247337, 'grow_policy': 1, 'learning_rate': 0.1993070315803166, 'max_depth': 3, 'min_child_weight': 7.0, 'n_estimators': 0, 'reg_alpha': 0.32213467759367453, 'reg_lambda': 0.8036297375321274, 'scale_pos_weight': 1.4841116094332476, 'subsample': 0.8064682844784123, '

In [22]:
# Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

X_test = test

X_test = X_test[X_train.columns]


In [24]:
# Indices
tree_method_options = ['auto', 'exact', 'approx', 'hist']
grow_policy_options = ['depthwise', 'lossguide']
n_estimators_options = [100, 200, 300, 400, 500]

# Reconstruir el modelo de XGBoost con los mejores hiperparámetros
best_model_xgb = xgb.XGBClassifier(
    n_estimators=n_estimators_options[best_xgb['n_estimators']],
    max_depth=best_xgb['max_depth'],
    learning_rate=best_xgb['learning_rate'],
    subsample=best_xgb['subsample'],
    colsample_bytree=best_xgb['colsample_bytree'],
    min_child_weight=best_xgb['min_child_weight'],
    gamma=best_xgb['gamma'],
    reg_alpha=best_xgb['reg_alpha'],  # L1 regularización
    reg_lambda=best_xgb['reg_lambda'],  # L2 regularización
    colsample_bylevel=best_xgb['colsample_bylevel'],  # Muestreo a nivel de split
    colsample_bynode=best_xgb['colsample_bynode'],  # Muestreo por nodo
    grow_policy=grow_policy_options[best_xgb['grow_policy']],  # Estrategia de crecimiento
    tree_method=tree_method_options[best_xgb['tree_method']],  # Método de construcción del árbol
    random_state=random_state,
    use_label_encoder=False,  # Evitar advertencias en versiones más recientes de XGBoost
    eval_metric="auc"  # Establecer AUC como métrica de evaluación
)

# Crear un nuevo pipeline reutilizando el preprocesador original y el mejor modelo
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', common_preprocessor),  # Reutilizamos el preprocesador definido previamente
    ('classifier', best_model_xgb),
])

# Entrenar el modelo con los datos de entrenamiento
xgb_pipeline.fit(X_train, y_train)

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['auction_age',
                                                   'auction_bidfloor',
                                                   'creative_height',
                                                   'creative_width',
                                                   'timezone_offset', 'mañana',
                                                   'tarde', 'noche',
                                                   'Desconocidos_auction_list_0_auction_list_0',
                                                   'IAB-5_auction_list_0',
                                                   'IAB1_auction_list_0',
                                                   'IAB1-1_auction_list_0',
                                                   'IAB1-2_auction_list_0',
                                                   'IAB1-3_aucti...
                               gpu_id=None, grow_policy='lossguide',
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.1993070315803166, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=3,
                               max_leaves=None, min_child_weight=7.0,
                               missing=nan, monotone_constraints=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, predictor=None,
                               random_state=43992294, ...))])

In [26]:
# Predecir en el conjunto de testeo
y_preds_xgb_test = xgb_pipeline.predict_proba(X_test)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_xgb_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros
file_name_xgb = (
    f"xgboost_preds_ne_{best_xgb['n_estimators']}_"
    f"md_{handle_none(best_xgb['max_depth'])}_"
    f"lr_{round(best_xgb['learning_rate'], 2)}_"
    f"ss_{round(best_xgb['subsample'], 2)}_"
    f"csb_{round(best_xgb['colsample_bytree'], 2)}_"
    f"cb_level_{round(best_xgb['colsample_bylevel'], 2)}_"
    f"cb_node_{round(best_xgb['colsample_bynode'], 2)}_"
    f"mcw_{round(best_xgb['min_child_weight'], 2)}_"
    f"gamma_{round(best_xgb['gamma'], 2)}_"
    f"ra_{round(best_xgb['reg_alpha'], 2)}_"
    f"rl_{round(best_xgb['reg_lambda'], 2)}_"
    f"spw_{round(best_xgb['scale_pos_weight'], 2)}_"
    f"gp_{grow_policy_options[best_xgb['grow_policy']]}_"
    f"tm_{tree_method_options[best_xgb['tree_method']]}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_xgb.to_csv(os.path.join("submits", file_name_xgb), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_xgb}")

/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 7, 8, 10, 12, 15, 16, 18, 21, 22, 24, 25, 29, 30, 33, 34, 35] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Archivo guardado en: submits/xgboost_preds_ne_0_md_3_lr_0.2_ss_0.81_csb_0.66_cb_level_0.78_cb_node_0.86_mcw_7.0_gamma_0.11_ra_0.32_rl_0.8_spw_1.48_gp_lossguide_tm_exact.csv
